In [3]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.linear_model import LogisticRegression, Ridge
from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor
from interpret import show
from pygam import LogisticGAM, LinearGAM
from gaminet import GAMINet
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, RobustScaler
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from load_datasets import load_adult_data, load_crimes_data
import tensorflow as tf
import torch


random_state = 1
task = 'classification'  # regression or classification

dataset, y_word_dict = load_adult_data()
dataset_name = 'adult'

X = pd.DataFrame(dataset['full']['X'])
y = np.array(dataset['full']['y'])
X, y = shuffle(X, y, random_state=random_state)

is_cat = np.array([dt.kind == 'O' for dt in X.dtypes])

num_cols = X.columns.values[~is_cat]
# one hot encoder pipeline replaced by pd.getdummies to make sure column names are concatenated
# cat_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))

# Handle unknown data as ignore
X = pd.get_dummies(X)
X = X.reindex(columns=X.columns, fill_value=0)
dummy_column_names = X.columns

# cat_pipe = Pipeline([cat_ohe_step])
num_pipe = Pipeline([('identity', FunctionTransformer())])  # , ('scaler', RobustScaler())])
transformers = [
    # ('cat', cat_pipe, cat_cols) replaced by pd.getdummies
    ('num', num_pipe, num_cols)
]
ct = ColumnTransformer(transformers=transformers, remainder='passthrough')
ct.fit(X)
X = ct.transform(X)

X = pd.DataFrame(X, columns=dummy_column_names)

scaler_dict = {}
for c in num_cols:


    scaler = RobustScaler()
    X[c] = scaler.fit_transform(X[c].values.reshape(-1, 1))
    scaler_dict[c] = scaler

# scaler = MinMaxScaler((0, 1))
    # scaler.fit([[0.0], [1.0]])
    # X[c] = scaler.transform(X[c].values.reshape(-1, 1))


def feature_importance_visualize(data_dict_global, folder="./results/", name="demo", save_png=False, save_eps=False):
    all_ir = []
    all_names = []
    for key, item in data_dict_global.items():
        if item["importance"] > 0:
            all_ir.append(item["importance"])
            all_names.append(key)

    max_ids = len(all_names)
    if max_ids > 0:
        fig = plt.figure(figsize=(0.4 + 0.6 * max_ids, 4))
        ax = plt.axes()
        ax.bar(np.arange(len(all_ir)), [ir for ir, _ in sorted(zip(all_ir, all_names))][::-1])
        ax.set_xticks(np.arange(len(all_ir)))
        ax.set_xticklabels([name for _, name in sorted(zip(all_ir, all_names))][::-1], rotation=60)
        plt.xlabel("Feature Name", fontsize=12)
        plt.ylim(0, np.max(all_ir) + 0.05)
        plt.xlim(-1, len(all_names))
        plt.title("Feature Importance")

        save_path = folder + name
        if save_eps:
            if not os.path.exists(folder):
                os.makedirs(folder)
            fig.savefig("%s.eps" % save_path, bbox_inches="tight", dpi=100)
        if save_png:
            if not os.path.exists(folder):
                os.makedirs(folder)
            fig.savefig("%s.png" % save_path, bbox_inches="tight", dpi=100)
    plt.show()


# %%

def make_plot(x, mean, upper_bounds, lower_bounds, feature_name, model_name, dataset_name, num_epochs='', debug=False):
    x = np.array(x)
    if debug:
        print("Num cols:", num_cols)
    if feature_name in num_cols:
        if debug:
            print("Feature to scale back:", feature_name)
        if feature_name == "capital.gain":
            pass # halt here
        x = scaler_dict[feature_name].inverse_transform(x.reshape(-1, 1)).squeeze()
    else:
        if debug:
            print("Feature not to scale back:", feature_name)

    plt.plot(x, mean, color='black')
    plt.fill_between(x, lower_bounds, mean, color='gray')
    plt.fill_between(x, mean, upper_bounds, color='gray')
    plt.xlabel(f'Feature value')
    plt.ylabel('Feature effect on model output')
    plt.title(f'Feature:{feature_name}')
    plt.savefig(f'plots/{model_name}_{dataset_name}_shape_{feature_name}_{num_epochs}epochs.pdf')
    plt.show()


# def plot_continuous_bar(
#     data_dict, feature_name, model_name, dataset_name, multiclass=False, show_error=True, title=None, xtitle="", ytitle=""
# ):
#     if feature_name == "capital.gain":
#         print("BUG")
#
#     if data_dict.get("scores", None) is None:  # pragma: no cover
#         return None
#
#     x_vals = data_dict["names"].copy()
#     y_vals = data_dict["scores"].copy()
#     y_hi = data_dict.get("upper_bounds", None)
#     y_lo = data_dict.get("lower_bounds", None)
#
#     # x_min = min(x_vals)
#     # x_max = max(x_vals)
#
#     if y_hi is None or multiclass:
#         show_error = False
#
#     def extend_x_range(x):
#         return x
#
#     def extend_y_range(y):
#         return np.r_[y, y[np.newaxis, -1]]
#
#     new_x_vals = extend_x_range(x_vals)
#     new_y_vals = extend_y_range(y_vals)
#     if show_error:
#         new_y_hi = extend_y_range(y_hi)
#         new_y_lo = extend_y_range(y_lo)
#
#     data = []
#     fill = "none"
#     if show_error:
#         fill = "tonexty"
#
#     if multiclass:
#         for i in range(y_vals.shape[1]):
#             class_name = (
#                 "Class {}".format(i)
#                 if "meta" not in data_dict
#                 else data_dict["meta"]["label_names"][i]
#             )
#             class_line = go.Scatter(
#                 x=new_x_vals,
#                 y=new_y_vals[:, i],
#                 line=dict(shape="hv"),
#                 name=class_name,
#                 mode="lines",
#             )
#             data.append(class_line)
#     else:
#         main_line = go.Scatter(
#             x=new_x_vals,
#             y=new_y_vals,
#             name="Main",
#             mode="lines",
#             line=dict(color="rgb(31, 119, 180)", shape="hv"),
#             fillcolor="rgba(68, 68, 68, 0.15)",
#             fill=fill,
#         )
#         data.append(main_line)
#
#     if show_error:
#         upper_bound = go.Scatter(
#             name="Upper Bound",
#             x=new_x_vals,
#             y=new_y_hi,
#             mode="lines",
#             marker=dict(color="#444"),
#             line=dict(width=0, shape="hv"),
#             fillcolor="rgba(68, 68, 68, 0.15)",
#             fill="tonexty",
#         )
#         lower_bound = go.Scatter(
#             name="Lower Bound",
#             x=new_x_vals,
#             y=new_y_lo,
#             marker=dict(color="#444"),
#             line=dict(width=0, shape="hv"),
#             mode="lines",
#         )
#         data = [lower_bound, main_line, upper_bound]
#
#     show_legend = True if multiclass or not show_error else False
#     layout = go.Layout(
#         title=title,
#         showlegend=show_legend,
#         xaxis=dict(title=xtitle),
#         yaxis=dict(title=ytitle),
#     )
#     yrange = None
#     if data_dict.get("scores_range", None) is not None:
#         scores_range = data_dict["scores_range"]
#         yrange = scores_range
#
#     main_fig = go.Figure(data=data, layout=layout)
#     main_fig.show()
#     main_fig.write_image(f'plots/{model_name}_{dataset_name}_shape_{feature_name}.pdf')


def make_plot_ebm(data_dict, feature_name, model_name, dataset_name, num_epochs='', debug=False):
    x_vals = data_dict["names"].copy()
    y_vals = data_dict["scores"].copy()

    # This is important since you do not plot plt.stairs with len(edges) == len(vals) + 1, which will have a drop to zero at the end
    y_vals = np.r_[y_vals, y_vals[np.newaxis, -1]]

    # This is the code interpretml also uses: https://github.com/interpretml/interpret/blob/2327384678bd365b2c22e014f8591e6ea656263a/python/interpret-core/interpret/visual/plot.py#L115

    # main_line = go.Scatter(
    #     x=x_vals,
    #     y=y_vals,
    #     name="Main",
    #     mode="lines",
    #     line=dict(color="rgb(31, 119, 180)", shape="hv"),
    #     fillcolor="rgba(68, 68, 68, 0.15)",
    #     fill="none",
    # )
    #
    # main_fig = go.Figure(data=[main_line])
    # main_fig.show()
    # main_fig.write_image(f'plots/{model_name}_{dataset_name}_shape_{feature_name}_{num_epochs}epochs.pdf')


    # This is my custom code used for plotting
    x = np.array(x_vals)
    if debug:
        print("Num cols:", num_cols)
    if feature_name in num_cols:
        if debug:
            print("Feature to scale back:", feature_name)
        x = scaler_dict[feature_name].inverse_transform(x.reshape(-1, 1)).squeeze()
    else:
        if debug:
            print("Feature not to scale back:", feature_name)

    plt.step(x, y_vals, where="post", color='black')
    # plt.fill_between(x, lower_bounds, mean, color='gray')
    # plt.fill_between(x, mean, upper_bounds, color='gray')
    plt.xlabel(f'Feature value')
    plt.ylabel('Feature effect on model output')
    plt.title(f'Feature:{feature_name}')
    plt.savefig(f'plots/{model_name}_{dataset_name}_shape_{feature_name}_{num_epochs}epochs.pdf')
    plt.show()



def make_plot_interaction(left_names, right_names, scores, feature_name_left, feature_name_right, model_name,
                          dataset_name):
    left_names = np.array(left_names)
    # print(right_names)
    if feature_name_left in num_cols:
        left_names = scaler_dict[feature_name_left].inverse_transform(left_names.reshape(-1, 1)).squeeze()
    right_names = np.array(right_names)
    if feature_name_right in num_cols:
        right_names = scaler_dict[feature_name_right].inverse_transform(right_names.reshape(-1, 1)).squeeze()
    fig, ax = plt.subplots()
    im = ax.pcolormesh(left_names, right_names, scores, shading='auto')
    fig.colorbar(im, ax=ax)
    plt.xlabel(feature_name_left)
    plt.ylabel(feature_name_right)
    plt.savefig(f'plots/{model_name}_{dataset_name}_interact_{feature_name_left.replace("?", "")} x {feature_name_right.replace("?", "")}.png')
    plt.show()


def make_one_hot_plot(class_zero, class_one, feature_name, model_name, dataset_name):
    plt.bar([0, 1], [class_zero, class_one], color='gray', tick_label=[f'{feature_name} = 0', f'{feature_name} = 1'])
    plt.ylabel('Feature effect on model output')
    plt.title(f'Feature:{feature_name}')
    print(feature_name)
    plt.savefig(f'plots/{model_name}_{dataset_name}_onehot_{str(feature_name).replace("?", "")}.pdf')
    plt.show()


# %%

def EBM_show(X, y):
    m4 = ExplainableBoostingRegressor(interactions=40, max_bins=256)
    m4.fit(X, y)
    ebm_global = m4.explain_global()
    show(ebm_global)


def EBM(X, y, dataset_name, model_name='EBM'):
    if task == "classification":
        ebm= ExplainableBoostingClassifier(interactions=50, max_bins=256)
    else:
        ebm = ExplainableBoostingRegressor(interactions=50, max_bins=256)
    ebm.fit(X, y)
    ebm_global = ebm.explain_global()

    for i in range(len(ebm_global.data()['names'])):
        data_names = ebm_global.data()
        print(data_names['names'][i])
        feature_name = data_names['names'][i]
        shape_data = ebm_global.data(i)

        if shape_data['type'] == 'interaction':
            x_name, y_name = feature_name.split(' x ')
            x_name = x_name.replace(' ', '')
            y_name = y_name.replace(' ', '')
            make_plot_interaction(shape_data['left_names'], shape_data['right_names'],
                                  np.transpose(shape_data['scores']),
                                  x_name, y_name, model_name, dataset_name)
            continue
        if len(shape_data['names']) == 2:
            make_one_hot_plot(shape_data['scores'][0], shape_data['scores'][1], feature_name, model_name, dataset_name)
        else:
            make_plot_ebm(shape_data, feature_name, model_name, dataset_name)
            # plot_continuous_bar(shape_data, feature_name, model_name, dataset_name)

    feat_for_vis = dict()
    for i, n in enumerate(ebm_global.data()['names']):
        feat_for_vis[n] = {'importance': ebm_global.data()['scores'][i]}
    feature_importance_visualize(feat_for_vis, save_png=True, folder='.', name='ebm_feat_imp')


# def modify_interaction_ranges(ebm_global, min_heatmap_val, max_heatmap_val):
#     for data_dict in ebm_global._internal_obj['specific']:
#         if data_dict['type'] == 'interaction':
#             data_dict['scores_range'] = (min_heatmap_val, max_heatmap_val)



def GAM(X, y, dataset_name, model_name='GAM'):
    if task == "classification":
        gam = LogisticGAM().fit(X, y)
    elif task == "regression":
        gam = LinearGAM().fit(X, y)
    for i, term in enumerate(gam.terms):
        if term.isintercept:
            continue
        XX = gam.generate_X_grid(term=i)
        pdep, confi = gam.partial_dependence(term=i, X=XX, width=0.95)
        # make_plot(XX[:,i].squeeze(), pdep, confi[:,0], confi[:,1], X.columns[i])
        if len(X[X.columns[i]].unique()) == 2:
            make_one_hot_plot(pdep[0], pdep[-1], X.columns[i], model_name, dataset_name)
        else:
            make_plot(XX[:, i].squeeze(), pdep, pdep, pdep, X.columns[i], model_name, dataset_name)

def Gaminet(X, y, dataset_name, model_name='Gaminet'):
    meta_info = {X.columns[i]: {'type': 'continuous'} for i in range(len(X.columns))}
    meta_info.update({'Y': {'type': 'target'}})

    # from sklearn.preprocessing import FunctionTransformer
    identity = FunctionTransformer()

    for i, (key, item) in enumerate(meta_info.items()):
        if item['type'] == 'target':
            continue
        # sx = MinMaxScaler((0, 1))
        # sx.fit([[0], [1]])
        # print(scaler_dict.keys())
        # print(X.columns)
        if key in scaler_dict:
            meta_info[key]['scaler'] = scaler_dict[key]
        else:
            meta_info[key]['scaler'] = identity

    if task == "classification":
        model_to_run = GAMINet(meta_info=meta_info, interact_num=20,
                               interact_arch=[40] * 5, subnet_arch=[40] * 5,
                               batch_size=1024, task_type="Classification", activation_func=tf.nn.relu,
                               main_effect_epochs=1000, interaction_epochs=1000, tuning_epochs=200,
                               # todo: main_effect_epochs=5000, interaction_epochs=5000, tuning_epochs=500,
                               lr_bp=[0.0001, 0.0001, 0.0001], early_stop_thres=[50, 50, 50],
                               heredity=True, loss_threshold=0.01, reg_clarity=1,
                               mono_increasing_list=[], mono_decreasing_list=[],  # the indices list of features
                               verbose=True, val_ratio=0.2, random_state=random_state)
        print(np.array(y).shape)
        model_to_run.fit(np.array(X), np.array(y).reshape(-1, 1))

    elif task == "regression":
        model_to_run = GAMINet(meta_info=meta_info, interact_num=20,
                               interact_arch=[40] * 5, subnet_arch=[40] * 5,
                               batch_size=1024, task_type="Regression", activation_func=tf.nn.relu,
                               main_effect_epochs=5000, interaction_epochs=5000, tuning_epochs=500,
                               lr_bp=[0.0001, 0.0001, 0.0001], early_stop_thres=[50, 50, 50],
                               heredity=True, loss_threshold=0.01, reg_clarity=1,
                               mono_increasing_list=[], mono_decreasing_list=[],  # the indices list of features
                               verbose=True, val_ratio=0.2, random_state=random_state)
        model_to_run.fit(np.array(X), np.array(y))

    data_dict = model_to_run.global_explain(save_dict=False, main_grid_size=1000)

    Xnames2Featurenames = dict(zip([X.columns[i] for i in range(X.shape[1])], X.columns))
    print(Xnames2Featurenames)

    for k in data_dict.keys():
        if data_dict[k]['type'] == 'pairwise':
            feature_name_left, feature_name_right = k.split('vs. ')
            feature_name_left = feature_name_left.replace(' ', '')
            feature_name_right = feature_name_right.replace(' ', '')
            make_plot_interaction(data_dict[k]['input1'], data_dict[k]['input2'], data_dict[k]['outputs'],
                                  Xnames2Featurenames[feature_name_left],
                                  Xnames2Featurenames[feature_name_right], model_name, dataset_name)
        elif data_dict[k]['type'] == 'continuous':
            # todo: reverse:
            # if len(X[Xnames2Featurenames[k]].unique()) == 2:
            #     make_one_hot_plot(data_dict[k]['outputs'][0], data_dict[k]['outputs'][-1],
            #                       Xnames2Featurenames[k], model_name, dataset_name)
            # else:
            try:
                make_plot(data_dict[k]['inputs'], data_dict[k]['outputs'], data_dict[k]['outputs'],
                          data_dict[k]['outputs'], Xnames2Featurenames[k], model_name, dataset_name)
            except:
                print("not continous")
        else:
            continue

    feat_for_vis = dict()
    for i, k in enumerate(data_dict.keys()):
        if 'vs.' in k:
            feature_name_left, feature_name_right = k.split('vs. ')
            feature_name_left = feature_name_left.replace(' ', '')
            feature_name_right = feature_name_right.replace(' ', '')
            feature_name_left = Xnames2Featurenames[feature_name_left]
            feature_name_right = Xnames2Featurenames[feature_name_right]
            feat_for_vis[f'{feature_name_left}\nvs.\n{feature_name_right}'] = {'importance': data_dict[k]['importance']}
        else:
            feat_for_vis[Xnames2Featurenames[k]] = {'importance': data_dict[k]['importance']}

    feature_importance_visualize(feat_for_vis, save_png=True, folder='.', name='gaminet_feat_imp')

def LR(X, y, dataset_name, model_name='LR'):
    m = Ridge()
    # if task == 'regression':
    # else:
    #     m = LogisticRegression()
    m.fit(X, y)
    import seaborn as sns
    #plot = sns.distplot(m.coef_)
    word_to_coef = dict(zip(m.feature_names_in_, m.coef_.squeeze()))
    dict(sorted(word_to_coef.items(), key=lambda item: item[1]))
    word_to_coef_df = pd.DataFrame.from_dict(word_to_coef, orient='index')
    print(word_to_coef_df)

    for i, feature_name in enumerate(X.columns):
        inp = torch.linspace(X[feature_name].min(), X[feature_name].max(), 1000)
        outp = word_to_coef[feature_name] *  inp #+ m.intercept_
        # outp = nam_model.feature_nns[i](inp).detach().numpy().squeeze()
        # if len(X[feature_name].unique()) == 2:
        #     make_one_hot_plot(outp[0], outp[-1], feature_name, model_name, dataset_name)
        # else:
        make_plot(inp, outp, outp, outp, feature_name, model_name, dataset_name)

# EBM_show(X, y) # for EBM_Show copy paste this script into a jupyter notebook and only run the EBM_Show dashboard
# EBM(X, y, dataset_name)
# GAM(X, y, dataset_name)
# Gaminet(X, y, dataset_name)
# LR(X, y, dataset_name)
# X.to_csv(f'export/X_full_{dataset_name}.csv')
# pd.DataFrame(y).to_csv(f'export/y_full_{dataset_name}.csv')

In [4]:
EBM_show(X, y) # for EBM_Show copy paste this script into a jupyter notebook and only run the EBM_Show dashboard
# EBM(X, y, dataset_name)
# GAM(X, y, dataset_name)
# Gaminet(X, y, dataset_name)
# LR(X, y, dataset_name)
# X.to_csv(f'export/X_full_{dataset_name}.csv')
# pd.DataFrame(y).to_csv(f'export/y_full_{dataset_name}.csv')

<!-- http://127.0.0.1:7001/1783941160432/ -->